# Inference_ Classifier

In [3]:
import pickle
import cv2
import mediapipe as mp
import numpy as np
import time
import tkinter as tk
from tkinter import Label
from threading import Thread

# Load the trained model
with open('./model.p', 'rb') as f:
    model_dict = pickle.load(f)
model = model_dict['model']

# Character labels
labels_dict = {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9',
               10: 'A', 11: 'B', 12: 'C', 13: 'D', 14: 'E', 15: 'F', 16: 'G', 17: 'H', 18: 'I',
               19: 'J', 20: 'K', 21: 'L', 22: 'M', 23: 'N', 24: 'O', 25: 'P', 26: 'Q', 27: 'R',
               28: 'S', 29: 'T', 30: 'U', 31: 'V', 32: 'W', 33: 'X', 34: 'Y', 35: 'Z'}

# MediaPipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

# GUI values
current_word = ""
sentence = ""
predicted_character_gui = ""

# GUI thread
def gui_loop():
    global current_word, sentence, predicted_character_gui
    root = tk.Tk()
    root.title("Word and Sentence Viewer")

    word_label = Label(root, text="Word: ", font=("Helvetica", 16), width=60, anchor='w')
    word_label.pack(padx=10, pady=10)

    sentence_label = Label(root, text="Sentence: ", font=("Helvetica", 16), width=60, anchor='w')
    sentence_label.pack(padx=10, pady=10)

    def update():
        word_label.config(text="Word: " + current_word)
        sentence_label.config(text="Sentence: " + sentence)
        root.after(200, update)

    update()
    root.mainloop()

# Start GUI
Thread(target=gui_loop, daemon=True).start()

# Webcam setup
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Failed to access camera.")
    exit()

stable_start_time = None
last_hand_time = time.time()
predicted_character = ""
prediction_printed = False

while True:
    hand_present = False
    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()
    if not ret or frame is None:
        print("Unable to read from camera.")
        break

    H, W, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        hand_present = True

        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_styles.get_default_hand_landmarks_style(),
                mp_styles.get_default_hand_connections_style()
            )

            for lm in hand_landmarks.landmark:
                x_.append(lm.x)
                y_.append(lm.y)

            if not x_ or not y_ or (max(x_) - min(x_) < 0.01) or (max(y_) - min(y_) < 0.01):
                continue

            for lm in hand_landmarks.landmark:
                norm_x = (lm.x - min(x_)) / (max(x_) - min(x_))
                norm_y = (lm.y - min(y_)) / (max(y_) - min(y_))
                data_aux.extend([norm_x, norm_y])

            x1 = int(min(x_) * W) - 20
            y1 = int(min(y_) * H) - 20
            x2 = int(max(x_) * W) + 20
            y2 = int(max(y_) * H) + 20

            try:
                prediction = model.predict([np.asarray(data_aux)])
                predicted_class = int(prediction[0])
                predicted_character = labels_dict.get(predicted_class, '?')

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                cv2.putText(frame, predicted_character, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)
            except Exception as e:
                print(f"Prediction error: {e}")
                continue

    if hand_present:
        if stable_start_time is None:
            stable_start_time = time.time()

        if time.time() - stable_start_time >= 4.0 and not prediction_printed:
            current_word += predicted_character
            predicted_character_gui = predicted_character
            prediction_printed = True
            last_hand_time = time.time()

    else:
        stable_start_time = None
        prediction_printed = False

    # Move word to sentence if hand is gone for 6 sec
    if not hand_present and (time.time() - last_hand_time >= 6) and current_word:
        sentence += current_word + " "
        current_word = ""

    # Show camera frame
    cv2.imshow("Sign Language Detection", frame)

    # Exit if ESC pressed
    if cv2.waitKey(1) & 0xFF == 27:
        break

# ✅ Save final sentence to file on exit
if sentence.strip():
    with open("output_sentences.txt", "a", encoding="utf-8") as file:
        file.write(sentence.strip() + "\n")

# Release camera
cap.release()
cv2.destroyAllWindows()
